In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.gridspec as gridspec
import pandas as pd
from tqdm.notebook import tqdm
import math
from IPython.display import clear_output


In [ ]:
possible_patterns=list(itertools.product([0,1,2],repeat=5)) #generates a list of the possible patterns for checks

In [ ]:
checkvecs_LUT=np.load('checkvecs.npy') #loads previously calculated lookup table

In [ ]:
dict1=pd.read_csv('d5l.csv',encoding='latin-1',header=None).drop_duplicates() #loads dictionary

In [ ]:
def indexof(string,char):
    indices = [i for i, c in enumerate(string) if c == char]
    return indices

In [ ]:
def check_alt(guess,solution):#Main word check function
    for i in guess:
        ocurrencias_guess=indexof(guess,i)
        ocurrencias_solution=indexof(solution,i)
        if len(ocurrencias_guess)<=len(ocurrencias_solution):
            if ocurrencias_guess==ocurrencias_solution:
                for j in ocurrencias_guess:
                    checkvec[j]=1
            else:
                coincidences=[j for j in ocurrencias_guess if j in ocurrencias_solution]
                notcoincidences=[j for j in ocurrencias_guess if j not in ocurrencias_solution]
                for j in coincidences:
                    checkvec[j]=1
                for j in notcoincidences:
                    checkvec[j]=2
        if len(ocurrencias_guess)>len(ocurrencias_solution):
            k=0
            coincidences=[j for j in ocurrencias_guess if j in ocurrencias_solution]
            notcoincidences=[j for j in ocurrencias_guess if j not in ocurrencias_solution]
            for j in coincidences:
                    checkvec[j]=1
                    k+=1
            for j in notcoincidences:
                if k<len(ocurrencias_solution):
                    checkvec[j]=2
                    k+=1
                else:
                    continue
    return checkvec
        
    

In [ ]:
def stats(dict1):
    letras='abcdefghijklmnopqrstuvwxyzñ'
    contador=np.zeros(len(letras),dtype=int)
    length=dict1.size
    for i in dict1[0]:
        for j in ''.join(set(i)):
            try:
                contador[letras.index(j)]+=i.count(j)
            except:
                continue
    return list(sorted(zip(letras,list(contador)),key=lambda x: x[1],reverse=True))
            
        

In [ ]:
# checkvecs_LUT=np.zeros((dict1.size,dict1.size,5))#checkvecs_LUT creation routine
# for i,guess in enumerate(tqdm(dict1[0])):
#     for j,word in enumerate (dict1[0]):
# #         print(guess,word)
#         checkvecs_LUT[i,j,:]= check_alt(guess,word)
# #         checkvecs_LUT= possible_guesses(guess,check(guess,word),dict1)[1]
# #         print(checkvecs_LUT)

In [ ]:
def rowindexof(word,dict1):
    return dict1.index[dict1[0]==word].tolist()

In [ ]:
def possibleguessesvec(word,checkvec,dict1,checkvecs_LUT):#Dictionary filter function. returns alist of possible words after a guess. checkvec is the vector after checking. Checkvec_LUT is the lookup table generated before.
    index=rowindexof(word,dict1)[0]
    possiblewords=[]
    for c,i in enumerate(checkvecs_LUT[index,:]):
        if (i==checkvec).all():
            possiblewords.append(dict1[0].iloc[c])
    return possiblewords

In [ ]:
def numberpossibleguessesvec(word,checkvec,dict1,checkvecs_LUT):#returns number of remaining words after a guess
    index=rowindexof(word,dict1)[0]
    possiblewords=0
    for c,i in enumerate(checkvecs_LUT[index,:]):
        if (i==checkvec).all():
            possiblewords+=1
    return possiblewords

In [ ]:
def exp_info_word(word,possible_patterns,dict1,checkvecs_LUT): #entropy calculation function
    accum=0
    for i in possible_patterns:
        px=numberpossibleguessesvec(word,i,dict1,checkvecs_LUT)/dict1.size
        if px>0:
            accum+=(px)*math.log2(1/px)
    return accum

In [ ]:
def step():
    print('Ingresar la palabra que pusiste en wordle \n Ingresá quit para salir')
    word=input()
    if word=='quit':
            return 0
    print('ingresá el vector que te devolvió \n (gris = 0, verde=1,amarillo =2, ejemplo: si te devolvió gris,gris,gris,amarillo,verde, el vector es 0,0,0,2,1')
    asd=input()
    if asd=='quit':
            return 0
    checkvec=list(map(int, asd.split(',')))
    return (word,checkvec)

In [ ]:
def game(dict1,checkvecs_LUT): # container function to solve steps in a loop
    word,checkvec=step()
    if word=='quit':
            return 0
    res=possibleguessesvec(word,checkvec,dict1,checkvecs_LUT)
    numb=numberpossibleguessesvec(word,checkvec,dict1,checkvecs_LUT)
    print(f'hay {numb} palabras posibles y son:')
    print(res)
    
    while True:
        word,checkvec=step()
        if word=='quit':
            break
        words=list(set(res).intersection(set(possibleguessesvec(word,checkvec,dict1,checkvecs_LUT)))).sort()
        if word in words:
            words.remove(word)
        numb=len(words)
        res=pd.Series(words)
        print(f'hay {numb} palabras posibles y son:')
        print(words)

In [ ]:
game(dict1,checkvecs_LUT)

In [ ]:
# new_dict=[]
# for i in dict1[0]:
#     if len(set(i))==5:
#         new_dict.append(i)

In [ ]:
# rank = pd.DataFrame([[1, 'b',0], [2, 'a',0]],

#                    columns=['index', 'word','information']) #entropy calculation routine
# for c,word in enumerate(tqdm(new_dict)):
    
#     info=exp_info_word(word,possible_patterns,dict1,checkvecs)
#     new=pd.DataFrame({'index': [c], 'word': [word],'information':[info]})
#     rank=pd.concat([rank,new])
#     rank=rank.sort_values(by=['information'],ascending=False)
#     print(rank.head(10))  
    
# rank.sort_values(by=['information'],ascending=False)    
# rank.head(10)

In [ ]:
# def check(guess,solution):#old check function
#     checkvec=[]
#     checkvec2=[]
#     checked_letters=[]
#     for i in guess:
#         if i not in checked_letters:
#             if i not in solution:
#                 checkvec.append(0)
#             elif guess.index(i)==solution.index(i):
#                 checkvec.append(1)
#             else: checkvec.append(2)
#         else: 
#             if i not in solution.replace(i,'-',checked_letters.count(i)):
#                 checkvec.append(0)
#             elif guess.replace(i,'-',checked_letters.count(i)).index(i)==solution.replace(i,'-',checked_letters.count(i)).index(i):
#                 checkvec.append(1)
#             else:
#                 checkvec.append(2) 
#         checked_letters.append(i)
    

#     return tuple(checkvec)

In [ ]:
# def possible_guesses(guess,checkvec,dict1):#old filter function
#         guessvector=zip(guess,checkvec)
#         sm=softmatches(guess,checkvec)
#         nm=nonmatches(guess,checkvec)
#         nm2=nm

#         for i in nm:
#             if i in sm or i in regexpgen(guess,checkvec):
#                 nm2=nm2.replace(i,'1')
#         possible_words= dict1[dict1[0].str.fullmatch(regexpgen(guess,checkvec))]
#         for i in sm:
#             possible_words= possible_words[possible_words[0].str.contains(i)]
#         for i in nm2:
#             possible_words= possible_words[~possible_words[0].str.contains(i)]
#   #print(f'Intentamos con {guess} \n Resultó en {checkvec} \n Las palabras posibles son \n {possible_words[0]} ')
#         return [possible_words,int(possible_words.shape[0])]






In [ ]:
# def regexpgen(guess,checkvec):#accesory function for possible_guesses
#   guessvector=zip(guess,checkvec)
#   regexp=''
#   for i,j in guessvector:
#     if j==0 or j==2:
#       regexp+='.'
#     elif j==1:
#       regexp+=i
#   return regexp
# regexpgen('adboo',(0,0,1,2,0))

In [ ]:
# def softmatches(guess,checkvec):#accesory function for possible_guesses
#   guessvector=zip(guess,checkvec)
#   regexp=''
#   for i,j in guessvector:
#     if j==2:
#       regexp+=i
     
#   return regexp

# softmatches('adboo',(0,1,2,0,0))

In [ ]:
# def nonmatches(guess,checkvec):#accesory function for possible_guesses
#     guessvector=zip(guess,checkvec)
#     regexp=''
#     for i,j in guessvector:
#       if j==0:
#        regexp+=i
     
  
#     if regexp=='':
#       regexp='1' 
  

     
#     return regexp
